In [ ]:
%pip install pandas elasticsearch

In [2]:
import pandas as pd
from elasticsearch import Elasticsearch

# CSV


send csv file to elastic


In [2]:
import pandas as pd
from elasticsearch import Elasticsearch

### Setup variables
csv_file_path = "data.csv"
es_host = "https://localhost:9200"  # Elasticsearch URL
index_name = "eurisko"  # Name of the index in Elasticsearch

## Model
### Read the CSV file
df = pd.read_csv(csv_file_path)
df = df.fillna("null")  # Replace NaN with 'null' or any other placeholder

### Convert DataFrame to JSON
records = df.to_dict(orient="records")

### Initialize Elasticsearch client
es = Elasticsearch([es_host])

### Index each record in Elasticsearch
for record in records:
    es.index(index=index_name, body=record)

## Output
### Print success message
print(f"Successfully indexed {len(records)} records to the '{index_name}' index.")

TlsError: TLS error caused by: TlsError(TLS error caused by: SSLError([SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:992)))

read from elastic with id filter


In [ ]:
### Setup variables
es_host = "http://localhost:9200"  # Elasticsearch URL
id_column = "ID"  # The column name for IDs in your data
filter_ids = [1, 2, 3]  # List of IDs you want to retrieve
index_name = "eurisko"  # Name of the index in Elasticsearch

## Model
### Initialize Elasticsearch client
es = Elasticsearch([es_host])

### Define the search query to match the IDs
query = {"query": {"terms": {id_column: filter_ids}}}

### Execute the search query
response = es.search(index=index_name, body=query)

### Extract the hits (matching documents)
hits = response["hits"]["hits"]

## Output
### Print the retrieved documents
for hit in hits:
    print(hit["_source"])

# vectors database


save vectors one at a time


In [39]:
## Input
### Import libraries
import numpy as np
import random
import time
from elasticsearch import Elasticsearch

### Setup variables
es_host = "http://localhost:9200"  # Elasticsearch URL
index_name = "vectorstore"  # Name of the index in Elasticsearch
vector_size = 50  # Size of each vector
num_vectors = 2000  # Number of vectors to generate
random_words = [
    "alpha",
    "beta",
    "gamma",
    "delta",
    "epsilon",
    "zeta",
    "eta",
    "theta",
    "iota",
    "kappa",
    "lambda",
    "mu",
    "nu",
    "xi",
    "omicron",
    "pi",
    "rho",
    "sigma",
    "tau",
    "upsilon",
]  # List of random words

## Model
### Generate random vectors
np.random.seed(0)  # For reproducibility
vectors = np.random.rand(num_vectors, vector_size).tolist()

### Initialize Elasticsearch client
es = Elasticsearch([es_host])

### Create index with mapping for dense_vector and name field
mapping = {
    "mappings": {
        "properties": {
            "vector": {"type": "dense_vector", "dims": vector_size},
            "name": {"type": "text"},  # Adding 'name' field
        }
    }
}
es.indices.create(
    index=index_name, body=mapping, ignore=400
)  # Ignore 400 if index already exists

starttime = time.time()
### Index each random vector with a random name
for i, vector in enumerate(vectors):
    random_name = " ".join(
        random.sample(random_words, 2)
    )  # Random combination of 2 words
    document = {"vector": vector, "name": random_name}
    es.index(index=index_name, body=document)
endtime = time.time()

## Output
### Print success message
print(
    f"Successfully indexed {num_vectors} random vectors with names into the '{index_name}' index in {endtime-starttime} seconds"
)

/tmp/ipykernel_28822/1385836427.py:53: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(


Successfully indexed 2000 random vectors with names into the 'vectorstore' index in 13.902277708053589 seconds


bulk save vectors


In [40]:
## Input
### Import libraries
import numpy as np
import pandas as pd
import time
from elasticsearch import Elasticsearch, helpers
import random

### Setup variables
es_host = "http://localhost:9200"  # Elasticsearch URL
index_name = "vectorstore"  # Name of the index in Elasticsearch
vector_size = 50  # Size of each vector
num_vectors = 2000  # Number of vectors to generate
random_words = [
    "alpha",
    "beta",
    "gamma",
    "delta",
    "epsilon",
    "zeta",
    "eta",
    "theta",
    "iota",
    "kappa",
    "lambda",
    "mu",
    "nu",
    "xi",
    "omicron",
    "pi",
    "rho",
    "sigma",
    "tau",
    "upsilon",
]
## Model
### Generate random vectors
np.random.seed(0)  # For reproducibility
vectors = np.random.rand(num_vectors, vector_size).tolist()

### Initialize Elasticsearch client
es = Elasticsearch([es_host])

### Create index with mapping for dense_vector
mapping = {
    "mappings": {
        "properties": {
            "vector": {"type": "dense_vector", "dims": vector_size},
            "name": {"type": "text"},  # Adding 'name' field
        }
    }
}
es.indices.create(
    index=index_name, body=mapping, ignore=400
)  # Ignore 400 if index already exists

### Prepare bulk data
actions = [
    {
        "_index": index_name,
        # "_id": i + 3000,
        "_source": {"vector": vector, "name": " ".join(random.sample(random_words, 2))},
    }
    for i, vector in enumerate(vectors)
]

starttime = time.time()
### Bulk index all vectors
helpers.bulk(es, actions)
endtime = time.time()


print(
    f"Successfully indexed {num_vectors} random vectors into the '{index_name}' index in {endtime-starttime} seconds"
)

/tmp/ipykernel_28822/1882706879.py:53: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(


Successfully indexed 2000 random vectors into the 'vectorstore' index in 0.4981706142425537 seconds


cosine similarity


In [27]:
### Setup variables
es_host = "http://localhost:9200"  # Elasticsearch URL
index_name = "vectorstore"  # Name of the index in Elasticsearch
vector_field = "vector"  # Field name where vectors are stored
query_vector = [0.7] * 50  # Example query vector (must match the dimensionality)
n = 5  # Number of top recommendations to retrieve

## Model
### Initialize Elasticsearch client
es = Elasticsearch([es_host])

### Define the search query for cosine similarity
query = {
    "size": n,
    "query": {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, doc['vector']) + 1.0",
                "params": {"query_vector": query_vector},
            },
        }
    },
    "_source": ["name", "vector"],  # Fetch 'name' and 'vector' fields in the response
}
starttime = time.time()

### Execute the search query
response = es.search(index=index_name, body=query)
endtime = time.time()

### Extract the hits (top recommendations)
hits = response["hits"]["hits"]

## Output
### Print the top `n` recommendations
for hit in hits:
    print(f"Name: {hit['_source']['name']}, Vector: {hit['_source']['vector']}")
print(f"Query executed in {endtime - starttime} seconds")

Name: pi iota, Vector: [0.8583639092841238, 0.3970260297012893, 0.30344747149584583, 0.47541434394037974, 0.5805702658610854, 0.8912380956098888, 0.3139102346282898, 0.6469437671307302, 0.4920824006727923, 0.7743974361641899, 0.4041689965269496, 0.7288626254013872, 0.27113076389372226, 0.6663518107013724, 0.6451225702859146, 0.5900478568087331, 0.5041734721617624, 0.6393502074514879, 0.5040971038994176, 0.19978437444490993, 0.3670625560839704, 0.10119631759147107, 0.6359936996780754, 0.595963404329483, 0.4333722525960275, 0.5108994089576718, 0.5467876170753613, 0.5478548696547741, 0.4015381426636617, 0.48671057428990994, 0.8445327535217351, 0.5306972579336255, 0.59478367954256, 0.7768408985615862, 0.24895984546680094, 0.6233116655126576, 0.5263209209182268, 0.5115966617048758, 0.3846984816804553, 0.9759036142526318, 0.3516412810131888, 0.8592079042791395, 0.311442398993132, 0.3476573755038592, 0.30273533668044106, 0.9509670856880753, 0.7533704208043165, 0.6026944956598547, 0.3958088499

/tmp/ipykernel_28822/822745206.py:29: ElasticsearchWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  response = es.search(index=index_name, body=query)


Hybrid search

In [38]:
### Setup variables
es_host = "http://localhost:9200"  # Elasticsearch URL
index_name = "vectorstore"  # Name of the index in Elasticsearch
vector_field = "vector"  # Field name where vectors are stored
query_vector = [0.7] * 50  # Example query vector (must match the dimensionality)
n = 5  # Number of top recommendations to retrieve
query_string = "theta"  # Replace with your search string

## Model
### Initialize Elasticsearch client
es = Elasticsearch([es_host])


## Model
### Create the Elasticsearch query
query = {
    "size": n,
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "name": query_string  # Match documents having the query string in 'name'
                    }
                },
                {
                    "script_score": {
                        "query": {"match_all": {}},  # Match all for scoring
                        "script": {
                            "source": "cosineSimilarity(params.query_vector, doc['vector']) + 1.0",
                            "params": {
                                "query_vector": query_vector
                            },  # Use cosine similarity for scoring
                        },
                    }
                },
            ]
        }
    },
    "_source": ["name", "vector"],  # Fetch 'name' and 'vector' fields in the response
}
starttime = time.time()

### Execute the search query
response = es.search(index=index_name, body=query)
endtime = time.time()

### Extract the hits (top recommendations)
hits = response["hits"]["hits"]

## Output
### Print the top `n` recommendations
for hit in hits:
    print(f"Name: {hit['_source']['name']}, Vector: {hit['_source']['vector']}")
print(f"Query executed in {endtime - starttime} seconds")

Name: eta theta, Vector: [0.47298460835621714, 0.8450377203404724, 0.7012593917069188, 0.5501599994504932, 0.6306174468372223, 0.31260326960579865, 0.8629243711948104, 0.6679400240831957, 0.9057781536019708, 0.8304087878553537, 0.4673182918443747, 0.42765671581615405, 0.7604498266623294, 0.09715887825978009, 0.6742383774039683, 0.49499200251932873, 0.5008010864995065, 0.8247210681695776, 0.3828639664018425, 0.8668037630800107, 0.5973436709331247, 0.9364509044300143, 0.11840104014193287, 0.7045995908866692, 0.3616379466387145, 0.8463014960773221, 0.8600278481886948, 0.698377827736145, 0.46820979182716416, 0.9315411043018724, 0.4925537131173531, 0.6866534828858862, 0.9326619097547095, 0.4082865347155108, 0.36714551731341094, 0.8393607394965934, 0.279368504881223, 0.8270532932911301, 0.16622914619912843, 0.7319463633904214, 0.03669929937616545, 0.5745888168959937, 0.6876740093362077, 0.8826380272978475, 0.9488541028081419, 0.8949426212316021, 0.670353892861211, 0.6333588478724245, 0.37167

/tmp/ipykernel_28822/2567639947.py:45: ElasticsearchWarning: The vector functions of the form function(query, doc['field']) are deprecated, and the form function(query, 'field') should be used instead. For example, cosineSimilarity(query, doc['field']) is replaced by cosineSimilarity(query, 'field').
  response = es.search(index=index_name, body=query)


In [ ]:


## Output
### Display the constructed query
print(query)

delete index


In [ ]:
## Input
### Import libraries
from elasticsearch import Elasticsearch

### Setup variables
es_host = "http://localhost:9200"  # Elasticsearch URL
index_name = "vectorstore"  # Name of the index you want to delete

## Model
### Initialize Elasticsearch client
es = Elasticsearch([es_host])

### Delete the index
try:
    response = es.indices.delete(
        index=index_name, ignore=[400, 404]
    )  # Ignore 400 and 404 errors if index does not exist
    print(f"Index '{index_name}' deleted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")